In [1]:
from datetime import datetime, date
import calendar
from time import mktime
import pandas, sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from pytz import utc, timezone

datafile_16 = "Resources/FY_2016_Dallas_Animal_Shelter_Data.csv"
datafile_17 = "Resources/FY_2017_Dallas_Animal_Shelter_Data.csv"
datafile_18 = "Resources/FY_2018_Dallas_Animal_Shelter_Data.csv"
datafile_19 = "Resources/FY_2019_Dallas_Animal_Shelter_Data.csv"
datafile_20 = "Resources/FY2020_Dallas_Animal_Shelter_Data.csv"
datafile_21 = "Resources/FY2021_Dallas_Animal_Shelter_Data.csv"

df_2016 = pd.read_csv(datafile_16)
df_2017 = pd.read_csv(datafile_17)
df_2018 = pd.read_csv(datafile_18)
df_2019 = pd.read_csv(datafile_19)
df_2020 = pd.read_csv(datafile_20)
df_2021 = pd.read_csv(datafile_21)

In [2]:
#MERGE ALL FISCAL YEAR DATA INTO ONE DATAFRAME
frames = [df_2016, df_2017, df_2018, df_2019, df_2020, df_2021]
dallas_shelter_df = pd.concat(frames)

In [3]:
dallas_shelter_df

,Animal Id,Animal Type,Animal Breed,Kennel Number,Kennel Status,Tag Type,Activity Number,Activity Sequence,Source Id,Census Tract,...,Service Request Number,Outcome Condition,Chip Status,Animal Origin,Additional Information,Month,Year,Unnamed: 16,Outcome Subtype,Unnamed: 27
0,A0000000,NaN,NaN,FREEZER,UNAVAILABLE,NaN,A16-017406,1,P0000000,701,...,NaN,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,UNABLE TO SCAN,FIELD,NaN,JUN.2016,FY2016,NaN,NaN,NaN
1,A0001656,DOG,LABRADOR RETR,LAB 01,LAB,NaN,NaN,1,P0003571,20500,...,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN,NaN,NaN
2,A0014681,DOG,PIT BULL,PAWS 24,UNAVAILABLE,NaN,NaN,1,P0721470,12302,...,50020,TREATABLE MANAGEABLE NON-CONTAGIOUS,SCAN NO CHIP,FIELD,TAGGHOUSEBROKEN/FREE ADOPT,FEB.2016,FY2016,NaN,NaN,NaN
3,A0117773,DOG,PIT BULL,LAB 01,LAB,NaN,NaN,1,P0062546,2200,...,NaN,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,SCAN NO CHIP,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN,NaN,NaN
4,A0119934,CAT,RUSSIAN BLUE,LOST,LOST REPORT,NaN,NaN,1,P0038200,75214,...,NaN,HEALTHY,UNABLE TO SCAN,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,A0701527,DOG,SCHNAUZER STAND,LFSD 03,IMPOUNDED,NaN,A21-259221,1,P0954698,12206,...,NaN,APP WNL,SCAN CHIP,FIELD,ITR,APR.2021,FY2021,NaN,WESTMORELD,NaN
11018,A1118116,CAT,DOMESTIC SH,N04,UNAVAILABLE,NaN,NaN,1,P0954696,7823,...,NaN,UNDERAGE,SCAN NO CHIP,OVER THE COUNTER,AEG,APR.2021,FY2021,NaN,UNDERAGE,NaN
11019,A1116166,DOG,GERM SHEPHERD,LOST,IMPOUNDED,NaN,NaN,0,P0951768,NaN,...,NaN,NaN,NaN,NaN,NaN,MAR.2021,FY2021,NaN,OTHER,NaN
11020,A1117299,DOG,PIT BULL,AD 003,AVAILABLE,NaN,NaN,1,P0900636,3901,...,NaN,APP WNL,SCAN NO CHIP,OVER THE COUNTER,JAS,APR.2021,FY2021,NaN,WESTMORELD,NaN


In [4]:
#GET RID OF DUPLICATES (SAME ANIMAL ID AND MONTH ADMITTED TO SHELTER)
dup_animals = dallas_shelter_df.loc[dallas_shelter_df.duplicated(subset = ['Animal Id', 'Month']), 'Animal Id'].unique()
dup_animal_id = pd.DataFrame(dup_animals)
dup_animal_id

,0
0,A0469113
1,A0489693
2,A0578637
3,A0660996
4,A0696700
...,...
6899,A1117310
6900,A1118114
6901,A1117945
6902,A1116461


In [5]:
dallas_shelter_clean = dallas_shelter_df[dallas_shelter_df['Animal Id'].isin(dup_animals)==False]
dallas_shelter_clean

,Animal Id,Animal Type,Animal Breed,Kennel Number,Kennel Status,Tag Type,Activity Number,Activity Sequence,Source Id,Census Tract,...,Service Request Number,Outcome Condition,Chip Status,Animal Origin,Additional Information,Month,Year,Unnamed: 16,Outcome Subtype,Unnamed: 27
0,A0000000,NaN,NaN,FREEZER,UNAVAILABLE,NaN,A16-017406,1,P0000000,701,...,NaN,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,UNABLE TO SCAN,FIELD,NaN,JUN.2016,FY2016,NaN,NaN,NaN
1,A0001656,DOG,LABRADOR RETR,LAB 01,LAB,NaN,NaN,1,P0003571,20500,...,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN,NaN,NaN
2,A0014681,DOG,PIT BULL,PAWS 24,UNAVAILABLE,NaN,NaN,1,P0721470,12302,...,50020,TREATABLE MANAGEABLE NON-CONTAGIOUS,SCAN NO CHIP,FIELD,TAGGHOUSEBROKEN/FREE ADOPT,FEB.2016,FY2016,NaN,NaN,NaN
3,A0117773,DOG,PIT BULL,LAB 01,LAB,NaN,NaN,1,P0062546,2200,...,NaN,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,SCAN NO CHIP,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN,NaN,NaN
4,A0119934,CAT,RUSSIAN BLUE,LOST,LOST REPORT,NaN,NaN,1,P0038200,75214,...,NaN,HEALTHY,UNABLE TO SCAN,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11014,A1100474,DOG,CHIHUAHUA SH,LFSD 18,UNAVAILABLE,NaN,A21-259249,1,P0954725,2000,...,NaN,APP WNL,SCAN CHIP,FIELD,AEG,APR.2021,FY2021,NaN,WESTMORELD,NaN
11017,A0701527,DOG,SCHNAUZER STAND,LFSD 03,IMPOUNDED,NaN,A21-259221,1,P0954698,12206,...,NaN,APP WNL,SCAN CHIP,FIELD,ITR,APR.2021,FY2021,NaN,WESTMORELD,NaN
11018,A1118116,CAT,DOMESTIC SH,N04,UNAVAILABLE,NaN,NaN,1,P0954696,7823,...,NaN,UNDERAGE,SCAN NO CHIP,OVER THE COUNTER,AEG,APR.2021,FY2021,NaN,UNDERAGE,NaN
11019,A1116166,DOG,GERM SHEPHERD,LOST,IMPOUNDED,NaN,NaN,0,P0951768,NaN,...,NaN,NaN,NaN,NaN,NaN,MAR.2021,FY2021,NaN,OTHER,NaN


In [6]:
#GET RID OF UNNECESSARY COLUMNS AND GET RID OF ERRORS
dallas_shelter_data_pre = dallas_shelter_clean.dropna(how='all', subset=['Animal Breed'])
dallas_shelter_data = dallas_shelter_data_pre.drop(columns=['Source Id', 'Activity Sequence', 'Receipt Number', 'Census Tract', 'Intake Time', 'Outcome Time','Tag Type', 'Activity Number', 'Service Request Number', 'Unnamed: 16','Unnamed: 27'])
dallas_shelter_data

,Animal Id,Animal Type,Animal Breed,Kennel Number,Kennel Status,Council District,Intake Type,Intake Subtype,Intake Total,Reason,...,Outcome Type,Outcome Date,Impound Number,Outcome Condition,Chip Status,Animal Origin,Additional Information,Month,Year,Outcome Subtype
1,A0001656,DOG,LABRADOR RETR,LAB 01,LAB,6,OWNER SURRENDER,GENERAL,1,DESTRUCTIVE AT HOME,...,EUTHANIZED,7/21/2016,K16-353536,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN
2,A0014681,DOG,PIT BULL,PAWS 24,UNAVAILABLE,7,OWNER SURRENDER,GENERAL,1,NaN,...,TRANSFER,3/31/2016,K16-338470,TREATABLE MANAGEABLE NON-CONTAGIOUS,SCAN NO CHIP,FIELD,TAGGHOUSEBROKEN/FREE ADOPT,FEB.2016,FY2016,NaN
3,A0117773,DOG,PIT BULL,LAB 01,LAB,2,OWNER SURRENDER,EUTHANASIA REQUESTED,1,NaN,...,EUTHANIZED,7/5/2016,K16-351923,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,SCAN NO CHIP,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN
4,A0119934,CAT,RUSSIAN BLUE,LOST,LOST REPORT,14,LOST REPORT,NaN,1,NaN,...,LOST REPORT,7/19/2016,K16-345201,HEALTHY,UNABLE TO SCAN,OVER THE COUNTER,NaN,JUL.2016,FY2016,NaN
5,A0132276,CAT,DOMESTIC SH,QC 03,RELEASED FROM QUARANTINE,10,CONFISCATED,QUARANTINE,1,NaN,...,RETURNED TO OWNER,9/27/2016,K16-359916,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,UNABLE TO SCAN,FIELD,NaN,SEP.2016,FY2016,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11014,A1100474,DOG,CHIHUAHUA SH,LFSD 18,UNAVAILABLE,1,OWNER SURRENDER,FIELD,1,MEDICAL,...,RETURNED TO OWNER,4/16/2021,K21-524674,APP WNL,SCAN CHIP,FIELD,AEG,APR.2021,FY2021,WESTMORELD
11017,A0701527,DOG,SCHNAUZER STAND,LFSD 03,IMPOUNDED,7,STRAY,CONFINED,1,OTHRINTAKS,...,RETURNED TO OWNER,4/16/2021,K21-524654,APP WNL,SCAN CHIP,FIELD,ITR,APR.2021,FY2021,WESTMORELD
11018,A1118116,CAT,DOMESTIC SH,N04,UNAVAILABLE,13,STRAY,OTC,1,OTHRINTAKS,...,FOSTER,4/16/2021,K21-524648,UNDERAGE,SCAN NO CHIP,OVER THE COUNTER,AEG,APR.2021,FY2021,UNDERAGE
11019,A1116166,DOG,GERM SHEPHERD,LOST,IMPOUNDED,NaN,STRAY,WEB,1,NaN,...,LOST EXP,4/16/2021,K21-522659,NaN,NaN,NaN,NaN,MAR.2021,FY2021,OTHER


In [7]:
#CHECK FORMATTING OF COLUMNS (DTYPES)
dallas_shelter_data['Intake Date'] = pd.to_datetime(dallas_shelter_data['Intake Date'])
dallas_shelter_data['Outcome Date'] = pd.to_datetime(dallas_shelter_data['Outcome Date'])
dallas_shelter_data.info()
#WRITE CLEAN MERGED DATA TO CSV
dallas_shelter_data.to_csv("Resources/DallasMergedClean.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172204 entries, 1 to 11020
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Animal Id               172204 non-null  object        
 1   Animal Type             172204 non-null  object        
 2   Animal Breed            172204 non-null  object        
 3   Kennel Number           172204 non-null  object        
 4   Kennel Status           172204 non-null  object        
 5   Council District        163774 non-null  object        
 6   Intake Type             172204 non-null  object        
 7   Intake Subtype          169098 non-null  object        
 8   Intake Total            144806 non-null  object        
 9   Reason                  97639 non-null   object        
 10  Staff Id                139216 non-null  object        
 11  Intake Date             172204 non-null  datetime64[ns]
 12  Due Out                 139216 

PermissionError: [Errno 13] Permission denied: 'Resources/DallasMergedClean.csv'

In [ ]:
#ADD A COLUMN TO DF TO MEASURE LENGTH OF TIME IN SHELTER PER ANIMAL
time_in_shelter = dallas_shelter_data["Outcome Date"] - dallas_shelter_data["Intake Date"]
dallas_shelter_data.insert(1, "Time in Shelter", time_in_shelter, True)
dallas_shelter_data

In [ ]:
#CHECK COLUMN DTYPES AGAIN FOR NEW COLUMN FORMAT
dallas_shelter_data.info()

In [ ]:
#GET SOME BASIC COUNTS TO GET A BETTER IDEA OF THE DATA
breeds = dallas_shelter_data["Animal Breed"].value_counts()
num_breeds = len(breeds)
animal_types = dallas_shelter_data["Animal Type"].value_counts()
num_animal_types = len(animal_types)
intake_types = dallas_shelter_data["Intake Type"].value_counts()
num_intake_types = len(intake_types)
origin_types = dallas_shelter_data["Animal Origin"].value_counts()
num_origin_types = len(origin_types)


print(f"Number of breeds: {num_breeds}")
print(f"Number of animal types: {num_animal_types}")
print(f"Number of intake types: {num_intake_types}")
print(f"Number of origin types: {num_origin_types}")

In [ ]:
#GET MORE INFO ABOUT DATA
print(f"Data Frame: \n{dallas_shelter_data}\n")
print(f"Column Types: \n{dallas_shelter_data.dtypes}\n")

animal_types_fulllist = dallas_shelter_data['Animal Type'].tolist()

animal_types_array = dallas_shelter_data['Animal Type'].to_numpy()


In [ ]:
#GET FULL LIST OF DIFFERNT POSSIBLE INPUTS IN COLUMNS
animal_types_list = dallas_shelter_data['Animal Type'].unique()
origin_types_list = dallas_shelter_data['Animal Origin'].unique()
intake_types_list = dallas_shelter_data['Intake Type'].unique()
intake_subtypes_list = dallas_shelter_data['Intake Subtype'].unique()
outcome_types_list = dallas_shelter_data['Outcome Type'].unique()
outcome_subtypes_list = dallas_shelter_data['Outcome Subtype'].unique()
intake_conditions_list = dallas_shelter_data['Intake Condition'].unique()
outcome_conditions_list = dallas_shelter_data['Outcome Condition'].unique()

print (f"Animal Types: {animal_types_list}")
print (f"Animal Origins: {origin_types_list}")
print (f"Intake Types: {intake_types_list}")
print (f"Intake Subtypes: {intake_subtypes_list}")
print (f"Intake Conditions: {intake_conditions_list}")
print (f"Outcome Types: {outcome_types_list}")
print (f"Outcome Subtypes: {outcome_subtypes_list}")
print (f"Outcome Conditions: {outcome_conditions_list}")

In [ ]:
#START GATHERING SUMMARY STATISTICS
groupby_animals = dallas_shelter_data.groupby(["Animal Type"])

totalAnimals = dallas_shelter_data["Animal Id"].nunique()
animalCount = groupby_animals["Animal Id"].nunique()
animalPercent = animalCount / totalAnimals * 100

shelter_animals = pd.DataFrame({"Total Animals": animalCount,
                                "Percent of Total": animalPercent})
shelter_animals["Percent of Total"] = shelter_animals["Percent of Total"].map("{:.2f}%".format)
shelter_animals[["Total Animals", "Percent of Total"]].sort_values(["Total Animals"], ascending=False)

In [ ]:
#START GATHERING SUMMARY STATISTICS
groupby_intakes = dallas_shelter_data.groupby(["Intake Type"])

intakeCount = groupby_intakes["Animal Id"].nunique()
intakePercent = intakeCount / totalAnimals * 100

shelter_intakes = pd.DataFrame({"Total Intakes": intakeCount,
                                "Percent of Total": intakePercent})
shelter_intakes["Percent of Total"] = shelter_intakes["Percent of Total"].map("{:.2f}%".format)
shelter_intakes[["Total Intakes", "Percent of Total"]].sort_values(["Total Intakes"], ascending=False)

In [ ]:
#START GATHERING SUMMARY STATISTICS
groupby_outcomes = dallas_shelter_data.groupby(["Outcome Type"])

outcomeCount = groupby_outcomes["Animal Id"].nunique()
outcomePercent = outcomeCount / totalAnimals * 100

shelter_outcomes = pd.DataFrame({"Total Outcomes": outcomeCount,
                                "Percent of Total": outcomePercent})
shelter_outcomes["Percent of Total"] = shelter_outcomes["Percent of Total"].map("{:.2f}%".format)
shelter_outcomes[["Total Outcomes", "Percent of Total"]].sort_values(["Total Outcomes"], ascending=False)

In [ ]:
#START GATHERING SUMMARY STATISTICS
groupby_year = dallas_shelter_data.groupby(["Year"])

yearCount = groupby_year["Animal Id"].nunique()
yearPercent = yearCount / totalAnimals * 100

shelter_years = pd.DataFrame({"Animals Per Year": yearCount,
                                "Percent of Total": yearPercent})
shelter_years["Percent of Total"] = shelter_years["Percent of Total"].map("{:.2f}%".format)
shelter_years[["Animals Per Year", "Percent of Total"]].sort_values(["Year"], ascending=False)

In [ ]:
#PLOT IDEA - BAR CHART 
years = dallas_shelter_data["Year"].value_counts()
x_axis = years.index.values

plt.bar(x_axis, yearCount, color='c', alpha=0.8, align='center')
plt.xlabel("Fiscal Year")
plt.ylabel("Animal Intakes")
plt.title("Number of Animal Intakes Per Year")
plt.xticks(rotation="vertical")
plt.show()



In [ ]:
#GET INFORMATION ON HOW LONG ANIMALS WERE IN SHELTER

bins = [pd.Timedelta("0 days"), pd.Timedelta("9 days"), pd.Timedelta("19 days"),pd.Timedelta("29 days"), pd.Timedelta("39 days"), pd.Timedelta("150 days")]
time_labels = ["<10", "10-19", "20-29", "30-39", "40+"]

dallas_shelter_data["Time in Shelter (Days)"] = pd.cut(dallas_shelter_data["Time in Shelter"], bins, labels=time_labels)
groupby_time = dallas_shelter_data.groupby(["Time in Shelter (Days)"])

animalCount = groupby_time["Animal Id"].nunique()
timePercent = animalCount / totalAnimals * 100

time_demographics = pd.DataFrame({"Animals": animalCount,
                                 "Percent of Animals": timePercent})

time_demographics["Percent of Animals"] = time_demographics["Percent of Animals"].map("{:.2f}%".format)

time_demographics


In [ ]:
#COMBINE LONG HAIR AND SHORT HAIR CHIHUAHUAS WITH CHIHUAHUAS, COMBINE ROUGH & SMOOTH COLLIES
dallas_shelter_data['Animal Breed'] = dallas_shelter_data['Animal Breed'].replace(['CHIHUAHUA SH','CHIHUAHUA LH','COLLIE SMOOTH','COLLIE ROUGH'],
                                                          ['CHIHUAHUA','CHIHUAHUA','COLLIE', 'COLLIE'])

#CREATE SEPARATE DATAFRAMES FOR EACH ANIMAL TYPE FOR MORE DETAILED ANALYSIS
dogs_df = dallas_shelter_data.loc[dallas_shelter_data["Animal Type"] == "DOG",:]
cats_df = dallas_shelter_data.loc[dallas_shelter_data["Animal Type"] == "CAT",:]
birds_df = dallas_shelter_data.loc[dallas_shelter_data["Animal Type"] == "BIRD",:]
livestock_df = dallas_shelter_data.loc[dallas_shelter_data["Animal Type"] == "LIVESTOCK",:]
wildlife_df = dallas_shelter_data.loc[dallas_shelter_data["Animal Type"] == "WILDLIFE",:]

In [ ]:
#GATHER INFORMATION ON HOW MANY OF EACH TYPE OF ANIMAL WERE ADMITTED TO SHELTERS
groupby_dogs_year = dogs_df.groupby(["Year"])
num_dogs_year = groupby_dogs_year["Animal Id"].nunique()

groupby_cats_year = cats_df.groupby(["Year"])
num_cats_year = groupby_cats_year["Animal Id"].nunique()

groupby_birds_year = birds_df.groupby(["Year"])
num_birds_year = groupby_birds_year["Animal Id"].nunique()

groupby_livestock_year = livestock_df.groupby(["Year"])
num_livestock_year = groupby_livestock_year["Animal Id"].nunique()

groupby_wildlife_year = wildlife_df.groupby(["Year"])
num_wildlife_year = groupby_wildlife_year["Animal Id"].nunique()


animals_per_year = pd.DataFrame({"Total Dogs Admitted": num_dogs_year,
                                "Total Cats Admitted": num_cats_year,
                                "Total Birds Admitted": num_birds_year,
                                "Total Livestock Admitted": num_livestock_year,
                                "Total Wildlife Admitted": num_wildlife_year,
                                "Total Animals Admitted": yearCount})

animals_per_year


In [ ]:
#LIST DOGS BREEDS TO SEARCH FOR ANY DUPLICATES OR SPELLING CHANGES BETWEEN YEARS
dogbreeds_type_list = dogs_df['Animal Breed'].unique()
dogbreeds_type_list

In [ ]:
groupby_dogbreed = dogs_df.groupby(['Animal Breed'])
num_dogs_breed = groupby_dogbreed["Animal Id"].nunique()
dogs_per_breed = pd.DataFrame({"Number of Breed": num_dogs_breed})

dogs_per_breed

In [ ]:
# MOST COMMON DOG TYPE FOUND IN SHELTER
dog_column = dogs_per_breed["Number of Breed"]
max_breed = dog_column.max()
index_max_breed = dog_column.idxmax()

print(max_breed)
print(index_max_breed)

In [ ]:
# TOP 10 MOST COMMON DOG BREEDS
dogs_per_breed.sort_values(by=['Number of Breed'], ascending=False).head(10)

In [ ]:
# GET YEARLY TRENDS FOR TOP TEN BREEDS PER FY
pitbulls_df = dogs_df.loc[dogs_df["Animal Breed"] == "PIT BULL",:]
groupby_pitbulls_year = pitbulls_df.groupby(["Year"])
num_pitbulls_year = groupby_pitbulls_year["Animal Id"].nunique()

chihuahuas_df = dogs_df.loc[dogs_df["Animal Breed"] == "CHIHUAHUA",:]
groupby_chihuahuas_year = chihuahuas_df.groupby(["Year"])
num_chihuahuas_year = groupby_chihuahuas_year["Animal Id"].nunique()

germshep_df = dogs_df.loc[dogs_df["Animal Breed"] == "GERM SHEPHERD",:]
groupby_germshep_year = germshep_df.groupby(["Year"])
num_germshep_year = groupby_germshep_year["Animal Id"].nunique()

lab_df = dogs_df.loc[dogs_df["Animal Breed"] == "LABRADOR RETR",:]
groupby_lab_year = lab_df.groupby(["Year"])
num_lab_year = groupby_lab_year["Animal Id"].nunique()

cairn_df = dogs_df.loc[dogs_df["Animal Breed"] == "CAIRN TERRIER",:]
groupby_cairn_year = cairn_df.groupby(["Year"])
num_cairn_year = groupby_cairn_year["Animal Id"].nunique()

rott_df = dogs_df.loc[dogs_df["Animal Breed"] == "ROTTWEILER",:]
groupby_rott_year = rott_df.groupby(["Year"])
num_rott_year = groupby_rott_year["Animal Id"].nunique()

husky_df = dogs_df.loc[dogs_df["Animal Breed"] == "ALASKAN HUSKY",:]
groupby_husky_year = husky_df.groupby(["Year"])
num_husky_year = groupby_husky_year["Animal Id"].nunique()

shihtzu_df = dogs_df.loc[dogs_df["Animal Breed"] == "SHIH TZU",:]
groupby_shihtzu_year = shihtzu_df.groupby(["Year"])
num_shihtzu_year = groupby_shihtzu_year["Animal Id"].nunique()

cattledog_df = dogs_df.loc[dogs_df["Animal Breed"] == "AUST CATTLE DOG",:]
groupby_cattledog_year = cattledog_df.groupby(["Year"])
num_cattledog_year = groupby_cattledog_year["Animal Id"].nunique()

boxer_df = dogs_df.loc[dogs_df["Animal Breed"] == "BOXER",:]
groupby_boxer_year = boxer_df.groupby(["Year"])
num_boxer_year = groupby_boxer_year["Animal Id"].nunique()


percent_total_pitbulls = num_pitbulls_year / num_dogs_year * 100
percent_total_chihuahuas = num_chihuahuas_year / num_dogs_year * 100
percent_total_germshep = num_germshep_year / num_dogs_year * 100
percent_total_lab = num_lab_year / num_dogs_year * 100
percent_total_cairn = num_cairn_year / num_dogs_year * 100
percent_total_rott = num_rott_year / num_dogs_year * 100
percent_total_husky = num_husky_year / num_dogs_year * 100
percent_total_shihtzu = num_shihtzu_year / num_dogs_year * 100
percent_total_cattledog = num_cattledog_year / num_dogs_year * 100
percent_total_boxer = num_boxer_year / num_dogs_year * 100


dog_breeds_per_year = pd.DataFrame({"Total Dogs Admitted": num_dogs_year,
                                "Total Pitbulls": num_pitbulls_year,
                                "Percent Pitbulls": percent_total_pitbulls,
                                "Total Chihuahuas": num_chihuahuas_year,
                                "Percent Chihuahuas": percent_total_chihuahuas,
                                "Total German Shepherds": num_germshep_year,
                                "Percent German Shepherds": percent_total_germshep,
                                "Total Labrador Retrievers": num_lab_year,
                                "Percent Labrador Retrievers": percent_total_lab,
                                "Total Cairn Terriers": num_cairn_year,
                                "Percent Cairn Terriers": percent_total_cairn,
                                "Total Rottweilers": num_rott_year,
                                "Percent Rottweilers": percent_total_rott, 
                                "Total Alaskan Huskies": num_husky_year,
                                "Percent Huskies": percent_total_husky,
                                "Total Shih Tzus": num_shihtzu_year,
                                "Percent Shih Tzus": percent_total_shihtzu,
                                "Total Cattledogs": num_cattledog_year,
                                "Percent Cattledogs": percent_total_cattledog,
                                "Total Boxers": num_boxer_year,
                                "Percent Boxers": percent_total_boxer,
                                "Total Animals Admitted": yearCount})

dog_breeds_per_year["Percent Pitbulls"] = dog_breeds_per_year["Percent Pitbulls"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Chihuahuas"] = dog_breeds_per_year["Percent Chihuahuas"].map("{:.2f}%".format)
dog_breeds_per_year["Percent German Shepherds"] = dog_breeds_per_year["Percent German Shepherds"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Labrador Retrievers"] = dog_breeds_per_year["Percent Labrador Retrievers"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Cairn Terriers"] = dog_breeds_per_year["Percent Cairn Terriers"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Rottweilers"] = dog_breeds_per_year["Percent Rottweilers"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Huskies"] = dog_breeds_per_year["Percent Huskies"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Shih Tzus"] = dog_breeds_per_year["Percent Shih Tzus"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Cattledogs"] = dog_breeds_per_year["Percent Cattledogs"].map("{:.2f}%".format)
dog_breeds_per_year["Percent Boxers"] = dog_breeds_per_year["Percent Boxers"].map("{:.2f}%".format)

dog_breeds_per_year

In [ ]:
# FIND AVERAGE LENGTH OF TIME EACH DOG BREED SPENT IN SHELTER

pitbulls_time = pitbulls_df.groupby(["Year"])["Time in Shelter"]
chihuahuas_time = chihuahuas_df.groupby(["Year"])["Time in Shelter"]
germshep_time = germshep_df.groupby(["Year"])["Time in Shelter"]
lab_time = lab_df.groupby(["Year"])["Time in Shelter"]
cairn_time = cairn_df.groupby(["Year"])["Time in Shelter"]
rott_time = rott_df.groupby(["Year"])["Time in Shelter"]
husky_time = husky_df.groupby(["Year"])["Time in Shelter"]
shihtzu_time = shihtzu_df.groupby(["Year"])["Time in Shelter"]
cattledog_time = cattledog_df.groupby(["Year"])["Time in Shelter"]
boxer_time = boxer_df.groupby(["Year"])["Time in Shelter"]
all_dogs_time = dogs_df.groupby(["Year"])["Time in Shelter"]

avg_pitbulls_year = pitbulls_time.apply(lambda x: np.mean(x))
avg_chihuahuas_year = chihuahuas_time.apply(lambda x: np.mean(x))
avg_germshep_year = germshep_time.apply(lambda x: np.mean(x))
avg_lab_year = lab_time.apply(lambda x: np.mean(x))
avg_cairn_year = cairn_time.apply(lambda x: np.mean(x))
avg_rott_year = rott_time.apply(lambda x: np.mean(x))
avg_husky_year = husky_time.apply(lambda x: np.mean(x))
avg_shihtzu_year = shihtzu_time.apply(lambda x: np.mean(x))
avg_cattledog_year = cattledog_time.apply(lambda x: np.mean(x))
avg_boxer_year = boxer_time.apply(lambda x: np.mean(x))
avg_dogs_time = all_dogs_time.apply(lambda x: np.mean(x))


In [ ]:
time_in_shelter_per_year = pd.DataFrame({"Average for All Dogs": avg_dogs_time,
                                    "Avg for Pitbulls": avg_pitbulls_year,
                                    "Avg for Chihuahuas": avg_chihuahuas_year,
                                    "Avg for German Shepherds": avg_germshep_year,
                                    "Avg for Labrador Retrievers": avg_lab_year,
                                    "Avg for Cairn Terriers": avg_cairn_year,
                                    "Avg for Rottweilers": avg_rott_year,
                                    "Avg for Alaskan Huskies": avg_husky_year,
                                    "Avg for Shih Tzus": avg_shihtzu_year,
                                    "Avg for Cattledogs": avg_cattledog_year,
                                    "Avg for Boxers": avg_boxer_year})

time_in_shelter_per_year

In [ ]:
API_petfinder_data = "Resources/Animals.csv"
petfinder_df_pre = pd.read_csv(API_petfinder_data)

In [ ]:
petfinder_df_pre.head(15)

In [ ]:
# CREATE A SEPARATE DATAFRAME FOR DALLAS ONLY

dallas_petfinder_df = petfinder_df_pre.loc[petfinder_df_pre["contact.address.city"] == "Dallas",:]
dallas_petfinder_df.head()

In [ ]:
# GET RID OF UNNCESSARY INFO FROM PETFINDER DATA

petfinder_df = petfinder_df_pre.drop(columns=['distance','animal_type', 'contact.address.postcode','animal_id', 'organization_id','attributes.declawed','colors.primary','attributes.house_trained','colors.secondary','colors.tertiary','attributes.spayed_neutered','contact.phone','attributes.shots_current','contact.address.address2', 'attributes.special_needs','contact.address.address1', 'environment.children','contact.email', 'contact.email', 'videos','environment.dogs','url', 'coat', 'contact.address.country', 'organization_id.1', 'primary_photo_cropped.small', 'primary_photo_cropped.medium','primary_photo_cropped.large', 'primary_photo_cropped.full', 'Unnamed: 0', 'tags', 'name', 'description', 'environment.cats','organization_animal_id','photos','primary_photo_cropped'])
petfinder_df['status_changed_at'] = pd.to_datetime(petfinder_df['status_changed_at'])

petfinder_df.head(10)

In [ ]:
#LOOK AT COLUMN DTYPES
dallas_petfinder_df.info()

In [ ]:
#ADD TIMESTAMP FOR WHEN DATA WAS EXTRACTED FROM PETFINDER
#extract_date = pd.Timestamp("2021-04-24 16:30:00").tz_localize("UTC")
#extract_date

In [ ]:
#ADD TIME ON PETFINDER AS COLUMN

#changed_at = dallas_petfinder_df["status_changed_at"].to_timestamp

#time_on_pf = extract_date - dallas_petfinder_df["status_changed_at"] # convert status changed to ts
#dallas_petfinder_df.insert(1, "Time on Petfinder", time_on_pf, True)
#dallas_petfinder_df.head()